In [1]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [2]:
x_path = './Data/train.csv'
X_train = pd.read_csv(x_path)

In [3]:
x_path = './Data/test.csv'
X_test = pd.read_csv(x_path)
X_test.shape

(418, 11)

In [4]:
Y_train = X_train['Survived']

Replace NA in Cabins column

In [5]:
X_train["Cabin"] = X_train["Cabin"].fillna('NA')
X_test["Cabin"] = X_test["Cabin"].fillna('NA')

Calculate number of letters in Cabin 

In [6]:
X_train['#Cabins'] = X_train['Cabin'].apply(lambda x: len(x))
X_test['#Cabins'] = X_test['Cabin'].apply(lambda x: len(x))

Get Titles

In [7]:
X_train['title'] = X_train.Name.apply(lambda x: x.split('.')[0].split(',')[1].strip())
# X_train["title"] = X_train["title"].fillna("NA")
X_test['title'] = X_test.Name.apply(lambda x: x.split('.')[0].split(',')[1].strip())
# X_test["title"] = X_test["title"].fillna("NA")

New Title

In [8]:
newtitles={
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"}

In [9]:
X_train['title']=X_train.title.map(newtitles)
X_test['title']=X_test.title.map(newtitles)

In [10]:
def newage (cols):
    title=cols[0]
    Sex=cols[1]
    Age=cols[2]
    if pd.isnull(Age):
        if title=='Master' and Sex=="male":
            return 4.57
        elif title=='Miss' and Sex=='female':
            return 21.8
        elif title=='Mr' and Sex=='male': 
            return 32.37
        elif title=='Mrs' and Sex=='female':
            return 35.72
        elif title=='Officer' and Sex=='female':
            return 49
        elif title=='Officer' and Sex=='male':
            return 46.56
        elif title=='Royalty' and Sex=='female':
            return 40.50
        else:
            return 42.33
    else:
        return Age 

In [11]:
X_train.Age=X_train[['title','Sex','Age']].apply(newage, axis=1)
X_test.Age=X_test[['title','Sex','Age']].apply(newage, axis=1)

In [12]:
X_test.shape

(418, 13)

In [13]:
X_train[X_train['PassengerId']==6]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,#Cabins,title
5,6,0,3,"Moran, Mr. James",male,32.37,0,0,330877,8.4583,NA,Q,2,Mr


Convert titles to numbers

In [14]:
X_train['title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Royalty', 'Officer'], dtype=object)

In [15]:
Titles = {'Mr':0, 'Mrs':1, 'Miss':2, 'Master':3, 'Royalty':4, 'Officer':5}
X_train['title'] = X_train['title'].apply(lambda x: Titles[x])
X_test['title'] = X_test['title'].apply(lambda x: Titles[x])

convert Age into bins

In [16]:
bins = [0, 18, 40, 50, 70,100]
names = ['0', '1', '2', '3','4']
X_train['AgeRange'] = pd.cut(X_train['Age'], bins, labels=names, include_lowest = True)
X_test['AgeRange'] = pd.cut(X_test['Age'], bins, labels=names, include_lowest = True)

In [17]:
X_train[X_train['PassengerId']==6]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,#Cabins,title,AgeRange
5,6,0,3,"Moran, Mr. James",male,32.37,0,0,330877,8.4583,NA,Q,2,0,1


Calculate survival priority

In [18]:
def sscore (cols):
    Sex=cols[0]
    AgeRange=cols[1]
    Parch=cols[2]
    if AgeRange=='0' and Parch == "0":
        return 1000
    elif AgeRange=='1' and Sex=="male": # 18-39
        return 1
    elif AgeRange=='2' and Sex=='male': #40-50
        return 1
    elif AgeRange=='0' and Sex=='male': #0-17
        return 100
    elif AgeRange=='3' and Sex=='male': #50-69
        return 10
    elif AgeRange=='4' and Sex=='male': #70 - 100
        return 10
    elif AgeRange=='1' and Sex=="female": #18-35
        return 10
    elif AgeRange=='2' and Sex=='female': #40-50
        return 100
    elif AgeRange=='0' and Sex=='female': #0-17
        return 100
    elif AgeRange=='3' and Sex=='female': #50-69
        return 10
    elif AgeRange=='4' and Sex=='female': #70 - 100
        return 10
    else:
        return 100 

In [19]:
X_train["Sscore"]=X_train[['Sex','AgeRange','Parch']].apply(sscore, axis=1)
X_test["Sscore"]=X_test[['Sex','AgeRange','Parch']].apply(sscore, axis=1)
X_test["Sscore"].shape

(418,)

New Title

In [20]:
def newage (cols):
    title=cols[0]
    Sex=cols[1]
    Age=cols[2]
    if pd.isnull(Age):
        if title=='Master' and Sex=="male":
            return 4.57
        elif title=='Miss' and Sex=='female':
            return 21.8
        elif title=='Mr' and Sex=='male': 
            return 32.37
        elif title=='Mrs' and Sex=='female':
            return 35.72
        elif title=='Officer' and Sex=='female':
            return 49
        elif title=='Officer' and Sex=='male':
            return 46.56
        elif title=='Royalty' and Sex=='female':
            return 40.50
        else:
            return 42.33
    else:
        return Age 

In [21]:
X_train.Age=X_train[['title','Sex','Age']].apply(newage, axis=1)
X_test.Age=X_test[['title','Sex','Age']].apply(newage, axis=1)

Is unaccompanied minor?

In [22]:
def unaccompanied_minors (cols):
    AgeRange=cols[0]
    Parch=cols[1]
    if AgeRange=='0' and Parch == "0":
        return 1
    else:
        return 0 

In [23]:
X_train["unaccompanied_minors"]=X_train[['AgeRange','Parch']].apply(unaccompanied_minors, axis=1)
X_test["unaccompanied_minors"]=X_test[['AgeRange','Parch']].apply(unaccompanied_minors, axis=1)
X_test.shape

(418, 16)

Calculate ticket length

In [24]:
X_train['Ticket2']=X_train.Ticket.apply(lambda x : len(x))
X_test['Ticket2']=X_test.Ticket.apply(lambda x : len(x))

In [25]:
X_train["Sex"] = X_train["Sex"].fillna("NA")
X_train["Embarked"] = X_train["Embarked"].fillna("NA")
X_train["Sex"] = X_train["Sex"].fillna("NA")
X_train["Embarked"] = X_train["Embarked"].fillna("NA")
X_train[['Pclass', 'SibSp', 'Fare']] = X_train[['Pclass', 'SibSp', 'Fare']].fillna(0)

In [26]:
X_test["Sex"] = X_test["Sex"].fillna("NA")
X_test["Embarked"] = X_test["Embarked"].fillna("NA")
X_test["Sex"] = X_test["Sex"].fillna("NA")
X_test["Embarked"] = X_test["Embarked"].fillna("NA")
X_test[['Pclass', 'Age', 'SibSp', 'Fare']] = X_test[['Pclass', 'Age', 'SibSp', 'Fare']].fillna(0)

Number of relatives

In [27]:
X_test["#Relatives"] = X_test['Parch'] + X_test['SibSp']
X_train["#Relatives"] = X_train['Parch'] + X_train['SibSp']

Gender and Embarked

In [28]:
genders = {'male': 0, 'female': 1, 'NA': 2}
embarks = {'C': 0, 'Q': 1, 'S': 2, 'NA': 3}
X_train['Sex'] = X_train['Sex'].apply(lambda x: genders[x])
X_train['Embarked'] = X_train['Embarked'].apply(lambda x: embarks[x])
X_test['Sex'] = X_test['Sex'].apply(lambda x: genders[x])
X_test['Embarked'] = X_test['Embarked'].apply(lambda x: embarks[x])

In [29]:
X_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,#Cabins,title,AgeRange,Sscore,unaccompanied_minors,Ticket2,#Relatives
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NA,2,2,0,1,1,0,9,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,0,3,1,1,10,0,8,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NA,2,2,2,1,10,0,16,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,2,4,1,1,10,0,6,1
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NA,2,2,0,1,1,0,6,0


Prep data for classification

In [30]:
X_train= X_train.drop(columns =['PassengerId', 'Cabin','Survived','Name', 'Ticket'] )
X_test= X_test.drop(columns =['Cabin','Name', 'Ticket'] )

In [31]:
trainX, testX, trainY, testY = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [32]:
trainX.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,#Cabins,title,AgeRange,Sscore,unaccompanied_minors,Ticket2,#Relatives
331,1,0,45.5,0,0,28.5000,2,4,0,2,1,0,6,0
733,2,0,23.0,0,0,13.0000,2,2,0,1,1,0,5,0
382,3,0,32.0,0,0,7.9250,2,2,0,1,1,0,17,0
704,3,0,26.0,1,0,7.8542,2,2,0,1,1,0,6,1
813,3,1,6.0,4,2,31.2750,2,2,2,0,100,0,6,6


# Classifiers

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import metrics

In [34]:
trainX[pd.isnull(trainX).any(axis=1)]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,#Cabins,title,AgeRange,Sscore,unaccompanied_minors,Ticket2,#Relatives


In [35]:

Ks = 20
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(trainX,trainY)
    yhat=neigh.predict(testX)
    mean_acc[n-1] = metrics.accuracy_score(testY, yhat)
  
    std_acc[n-1]=np.std(yhat==testY)/np.sqrt(yhat.shape[0])

mean_acc

array([0.77653631, 0.77094972, 0.81005587, 0.76536313, 0.7877095 ,
       0.7877095 , 0.78212291, 0.76536313, 0.77653631, 0.75418994,
       0.74301676, 0.75418994, 0.76536313, 0.77094972, 0.76536313,
       0.75977654, 0.77094972, 0.77094972, 0.76536313])

In [36]:
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"
#          ,"XGBClassifier"
        ]

classifiers = [
    KNeighborsClassifier(6),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
#     MLPClassifier(alpha=1,hidden_layer_sizes=(20,50,13),max_iter=10000,activation ='logistic',
#                     solver='adam',learning_rate_init=0.001),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
#     ,XGBClassifier(max_depth=10, learning_rate=0.1, n_estimators=1000, nthread=50)
]

In [37]:
for name, clf in zip(names, classifiers):
        clf.fit(trainX, trainY)
        #score = clf.score(testX, testY)
        scores_rf=cross_val_score(clf,trainX,trainY,scoring='accuracy',cv=5)
        print(name,np.mean(scores_rf))

Nearest Neighbors 0.745744995190605
Linear SVM 0.8020258834163538
RBF SVM 0.6404738004977741
Gaussian Process 0.7865524845746602
Decision Tree 0.7948838794808228
Random Forest 0.804793398590222
Neural Net 0.7978696161495082
AdaBoost 0.8018569084496564
Naive Bayes 0.7949044163308473
QDA 0.5005147484620065


C:\Users\anand\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\anand\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\anand\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\anand\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\anand\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\anand\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\anand\Anaconda3\lib\site-

Get data to predict

In [38]:
test_df= X_test.drop(columns =['PassengerId'] )
test_df.columns
test_df[pd.isnull(test_df).any(axis=1)]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,#Cabins,title,AgeRange,Sscore,unaccompanied_minors,Ticket2,#Relatives


In [39]:
Final_df=pd.DataFrame()

In [40]:
Final_df["PassengerId"]=X_test["PassengerId"]

In [41]:
for name, clf in zip(names, classifiers):
        clf.fit(trainX, trainY)
        #score = clf.score(testX, testY)
        final_predictions = clf.predict(test_df)
        Final_df[name] = final_predictions
#         submission = pd.DataFrame({
#         "PassengerId": X_test["PassengerId"],
#         "Survived": final_predictions
#         })
#         submission.to_csv(name+'_submission.csv', index=False)
        scores_rf=cross_val_score(clf,trainX,trainY,scoring='accuracy',cv=5)
        print(name,np.mean(scores_rf))

Nearest Neighbors 0.745744995190605
Linear SVM 0.8020258834163538
RBF SVM 0.6404738004977741
Gaussian Process 0.7865524845746602
Decision Tree 0.7920866766836201
Random Forest 0.8188781850454012
Neural Net 0.7426704750648412
AdaBoost 0.8018569084496564
Naive Bayes 0.7949044163308473
QDA 0.5005147484620065


C:\Users\anand\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\anand\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\anand\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\anand\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\anand\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\anand\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\anand\Anaconda3\lib\site-

In [50]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
svc=make_pipeline(StandardScaler(),SVC(random_state=1))
r=[0.0001,0.001,0.1,1,10,50,100]
PSVM=[{'svc__C':r, 'svc__kernel':['linear']},
      {'svc__C':r, 'svc__gamma':r, 'svc__kernel':['rbf']}]
GSSVM=GridSearchCV(estimator=svc, param_grid=PSVM, scoring='accuracy', cv=2)

In [52]:
model=GSSVM.fit(trainX, trainY)

In [54]:
Final_df['GSSVM']=model.predict(test_df)

In [55]:
Final_df.columns

Index(['PassengerId', 'Nearest Neighbors', 'Linear SVM', 'RBF SVM',
       'Gaussian Process', 'Decision Tree', 'Random Forest', 'Neural Net',
       'AdaBoost', 'Naive Bayes', 'QDA', 'Sums', 'Sum1', 'GSSVM'],
      dtype='object')

In [56]:
Final_df['Sums'] = Final_df['Nearest Neighbors']+Final_df['Linear SVM']+Final_df['RBF SVM']+Final_df['Gaussian Process']+Final_df['Decision Tree']+Final_df['Random Forest']+Final_df['Neural Net']+Final_df['AdaBoost']+Final_df['Naive Bayes']+Final_df['QDA'] +Final_df['GSSVM']

In [57]:
Final_df.head()

,PassengerId,Nearest Neighbors,Linear SVM,RBF SVM,Gaussian Process,Decision Tree,Random Forest,Neural Net,AdaBoost,Naive Bayes,QDA,Sums,Sum1,GSSVM
0,892,0,0,0,0,0,0,0,0,0,0,0,0,0
1,893,1,1,0,0,0,0,1,1,1,0,6,0,1
2,894,0,0,0,0,0,0,0,0,0,1,1,0,0
3,895,0,0,1,0,0,0,0,0,0,0,1,0,0
4,896,1,1,0,1,1,1,1,1,1,0,9,1,1


In [58]:
bins = [0, 5,10]
Sums = ['0', '1']
Final_df['Sum1'] = pd.cut(Final_df['Sums'], bins, labels=Sums, include_lowest = True)

In [59]:
Final_df.head()
Final_df.to_csv('Combined_data.csv', index=False)

In [61]:
submission = pd.DataFrame({
        "PassengerId": Final_df["PassengerId"],
        #"Survived": Final_df["Sum1"]
        "Survived": Final_df['GSSVM']
    })
submission.to_csv('Combined_submission.csv', index=False)